In [ ]:
import pandas as pd
import numpy as np
from google.colab import drive
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
drive.mount('/content/gdrive')

In [ ]:
raw_data = pd.read_csv("/content/gdrive/My Drive/export.csv", header = None)

In [ ]:
uniq_names = raw_data[4].unique()
orderbooks = {}
for i in uniq_names:
    orderbooks[i] = raw_data[raw_data[4] == i]
    orderbooks[i].sort_values(3,inplace = True)
  

In [ ]:
#computing St
for i in orderbooks.keys():
  new_col = orderbooks[i][6]*orderbooks[i][5]/(orderbooks[i][6]+orderbooks[i][8])
  new_col += +orderbooks[i][8]*orderbooks[i][7]/(orderbooks[i][6]+orderbooks[i][8])
  orderbooks[i]['St'] = new_col

In [ ]:
def write_agrs(t_span, names = orderbooks.keys()):
  agr_orderbooks = {}
  t_intervals = [(t,t+t_span) for t in range(min(raw_data[3]),max(raw_data[3])-t_span, t_span)]
  for i in names:
#    print(i)
    buffer_dict={}
    buffer_dict['intervals'] = []
    buffer_dict['mins'] = []
    buffer_dict['maxs'] = []
    buffer_dict['open'] = []
    buffer_dict['close'] = []
    buffer_dict['rets'] = []
    buffer_dict['mean'] = []
    buffer_dict['std'] = []
    buffer_dict['num_changes'] = []
    for ts in t_intervals:
      vals_for_period = orderbooks[i][(orderbooks[i][3]>=ts[0]) & (ts[1]>orderbooks[i][3])]
      try:
        mins = min(vals_for_period['St'])
        maxs = max(vals_for_period['St'])
        opens = vals_for_period['St'].iloc[0]
        closes = vals_for_period['St'].iloc[-1]
        returns = closes/opens - 1
        stds = np.std(vals_for_period['St'])
        means = np.mean(vals_for_period['St'])
      except:
        mins = maxs = opens = closes = means = stds = returns = None
      buffer_dict['intervals'].append(ts[1])
      buffer_dict['maxs'].append(maxs)
      buffer_dict['mins'].append(mins)
      buffer_dict['open'].append(opens)
      buffer_dict['close'].append(closes)
      buffer_dict['rets'].append(returns)
      buffer_dict['mean'].append(means)
      buffer_dict['std'].append(stds)
      buffer_dict['num_changes'].append(len(vals_for_period))
    agr_orderbooks[i] = pd.DataFrame.from_dict(buffer_dict)
  return agr_orderbooks
    

In [ ]:
agr_orderbooks_5min = write_agrs(300000)

returns = {}
for i in agr_orderbooks_5min.keys():
  returns[i] = (agr_orderbooks_5min[i]['rets']).to_list()
dfreturns = pd.DataFrame.from_dict(returns)

corr_matrix = dfreturns.corr()
most_correlated_5mins = []
least_correlated_5mins = []
j = 0
for i in corr_matrix.columns:
  least_correlated_5mins.append((i,corr_matrix.columns[corr_matrix[i].argmin()],corr_matrix[i].min()))
  corr_matrix[i].iloc[j] = 0
  j+=1
  most_correlated_5mins.append((i,corr_matrix.columns[corr_matrix[i].argmax()],corr_matrix[i].max()))

In [ ]:
agr_orderbooks_1min = write_agrs(60000)

returns = {}
for i in agr_orderbooks_1min.keys():
  returns[i] = (agr_orderbooks_1min[i]['rets']).to_list()
dfreturns = pd.DataFrame.from_dict(returns)

corr_matrix = dfreturns.corr()
most_correlated_1mins = []
least_correlated_1mins = []
j = 0
for i in corr_matrix.columns:
  least_correlated_1mins.append((i,corr_matrix.columns[corr_matrix[i].argmin()],corr_matrix[i].min()))
  corr_matrix[i].iloc[j] = 0
  j+=1
  most_correlated_1mins.append((i,corr_matrix.columns[corr_matrix[i].argmax()],corr_matrix[i].max()))

In [ ]:
returns = {}
for i in agr_orderbooks_1min.keys():
  returns[i] = (agr_orderbooks_1min[i]['rets']).to_list()
dfreturns = pd.DataFrame.from_dict(returns)

corr_matrix = dfreturns.corr()
more95corr = []
for i in corr_matrix.columns:
  ka = []
  for j in range(len(corr_matrix)):
    if 1>corr_matrix[i].iloc[j]>0.95:
     # print(i,corr_matrix.columns[j])
      ka.append(j)
  for j in ka:
    more95corr.append((i,corr_matrix.columns[j]))
more95corr = ([frozenset(j) for j in more95corr])
more95corr = list(set(map(tuple,more95corr)))
print(more95corr)

In [ ]:
for i in more95corr:
  print('в', i[0],'совершается в среднем в',
        np.mean(agr_orderbooks_5min[i[0]]['num_changes'])/np.mean(agr_orderbooks_5min[i[1]]['num_changes']),
        'раз больше транзакций, чем в', i[1], 'при',np.mean(agr_orderbooks_5min[i[0]]['num_changes'])/300,'тр/c в',i[0])
names_to_check = []
for i in more95corr:
  if i[0] not in names_to_check:
    names_to_check.append(i[0])
  if i[1] not in names_to_check:
    names_to_check.append(i[1])

In [ ]:
agr_orderbooks_10secs = write_agrs(10000, names = names_to_check)

In [ ]:
agr_orderbooks_5secs = write_agrs(5000, names = names_to_check)

In [ ]:
agr_orderbooks_2secs = write_agrs(2000, names = names_to_check)

# Первичный рынок vs вторичный

In [ ]:
pairs = [('BTCBUSD','ETHUSDT')]#,('ETHUSDT_210625','ETHUSDT'),('BTCBUSD','BTCUSDT_210625'),('BTCUSDT','BTCUSDT_210625')]

for p in pairs:
  one = agr_orderbooks_5secs[p[0]]
  two = agr_orderbooks_5secs[p[1]]
  one = one[(one['num_changes']>0)&(one['rets']!=0)]
  two = two[(two['num_changes']>0)&(two['rets']!=0)]
  res = two.merge(one,how='inner',on=['intervals'], suffixes=['_two','_one'])[['rets_one','rets_two']]

  a = (res['rets_two'].iloc[1:-1])
  b = res['rets_one'].iloc[2:]
  a1 = (res['rets_two'].iloc[1:-1])/(res['rets_two'].iloc[1:-1]).apply(lambda x: abs(x))
  b1 = res['rets_one'].iloc[2:]/(res['rets_one'].iloc[2:]).apply(lambda x: abs(x))
  sum = 0
  for i in range(0,len(a1)):
    sum+=a1.iloc[i]*b1.iloc[i]
  print(p[1],'=>',p[0])
  print(sum/len(a1),np.corrcoef(a1,b1)[0,1],np.corrcoef(a,b)[0,1])
  print("volatility fraction:", np.std(a)/np.std(b))

  a = (res['rets_one'].iloc[1:-1])
  b = res['rets_two'].iloc[2:]
  a1 = (res['rets_one'].iloc[1:-1])/(res['rets_one'].iloc[1:-1]).apply(lambda x: abs(x))
  b1 = res['rets_two'].iloc[2:]/(res['rets_two'].iloc[2:]).apply(lambda x: abs(x))
  sum = 0
  for i in range(0,len(a1)):
    sum+=a1.iloc[i]*b1.iloc[i]

  print(p[0],'=>',p[1])
  print(sum/len(a1),np.corrcoef(a1,b1)[0,1],np.corrcoef(a,b)[0,1])
  print("volatility fraction:", np.std(a)/np.std(b),'\n')

# Автокорреляция

In [ ]:
insts = ['BTCBUSD','BTCUSDT','ETHUSDT_210625','ETHUSDT','BTCUSDT_210625']
vals = {}
for p in insts:
  one = agr_orderbooks_5min[p]
  vals[p] = []
  one = one[(one['num_changes']>0)&(one['rets']!=0)]
  for i in range(1,5):
    a = one['rets'].iloc[:-i]
    b = one['rets'].iloc[i:]
    vals[p].append(np.corrcoef(a,b)[0,1])
    #print(p,':',np.corrcoef(a,b)[0,1])


In [ ]:
n = len(insts)
i = 0
print(n)
fig, ax = plt.subplots(n)
fig.set_size_inches(5,4*n)
for p in insts:
  ax[i].bar(range(1,5), vals[p])
  ax[i].set_title(p)
  ax[i].grid()
  i+=1

# Отскок

In [ ]:
bid_meanvols = {}
bid_stdvols = {}
ask_meanvols = {}
ask_stdvols = {}
for p in insts:
  bid_meanvols[p] = np.mean(orderbooks[p][6]) 
  bid_stdvols[p] = np.std(orderbooks[p][6])
  ask_meanvols[p] = np.mean(orderbooks[p][8]) 
  ask_stdvols[p] = np.std(orderbooks[p][8])


In [ ]:
#вероятности для бида + средний экстремум
Pr = {}
dextr = {}
for p in insts:
  Pr[p] = []
  dextr[p] = {'mean':[],'std':[], 'median':[]}
  for k in np.arange(0,3,0.5):
    pn = 0
    p0 = 0
    delta_extremums = []
    for i in range(len(orderbooks[p])-1):
      if orderbooks[p][6].iloc[i] >= bid_meanvols[p]+k*bid_stdvols[p]:
        j = i+1
        #orderbooks[p][5].iloc[i]
        while orderbooks[p][5].iloc[j]==orderbooks[p][5].iloc[i] and j<len(orderbooks[p])-1:
          j+=1
        if orderbooks[p][5].iloc[j]>orderbooks[p][5].iloc[i]:
          pn+=1
          while orderbooks[p][5].iloc[j]>=orderbooks[p][5].iloc[j-1] and j<len(orderbooks[p])-2:
            j+=1
          delta_extremums.append(orderbooks[p][5].iloc[j-1]/orderbooks[p][5].iloc[i]-1)
        else:
          p0+=1
    dextr[p]['mean'].append(np.mean(delta_extremums))
    dextr[p]['std'].append(np.std(delta_extremums))
    dextr[p]['median'].append(np.median(delta_extremums))
    Pr[p].append(pn/(p0+pn))
    print(p,k,":",pn/(p0+pn), dextr[p]['mean'][-1])

In [ ]:
#доля плотных объемов во всем датасете
count_vols = {}
for p in insts:
  count_vols[p]=[]
  for k in np.arange(0,3,0.5):
      count_vols[p].append(len(orderbooks[p][6][orderbooks[p][6]>=bid_meanvols[p]+k*bid_stdvols[p]])/len(orderbooks[p][6]))
      print(p,k,':',len(orderbooks[p][6][orderbooks[p][6]>=bid_meanvols[p]+k*bid_stdvols[p]])/len(orderbooks[p][6]))

# Графики (бид)

первый график - вероятность отскока на одно ценовое деление

второй - как далеко в среднем происходит отскок

третий - то же самое, но стандартное отклонение

четвертый - то же самое, но медиана

пятый - доля плотных уровней при разных k

In [ ]:
#вероятности
i = 0
fig, ax = plt.subplots(len(insts),5)
fig.set_size_inches(25,4*len(insts))
for p in insts:
  ax[i][0].plot(np.arange(0,3,0.5), Pr[p])
  ax[i][1].plot(np.arange(0,3,0.5), dextr[p]['mean'])
  ax[i][2].plot(np.arange(0,3,0.5), dextr[p]['std'])
  ax[i][3].plot(np.arange(0,3,0.5), dextr[p]['median'])
  ax[i][4].plot(np.arange(0,3,0.5), count_vols[p])
  ax[i][0].set_title(p+' probability')
  ax[i][1].set_title(p+' mean extr')
  ax[i][2].set_title(p+' std extr')
  ax[i][3].set_title(p+' median extr')
  ax[i][4].set_title(p+' vol ratio')
  ax[i][0].grid()
  ax[i][1].grid()
  ax[i][2].grid()
  ax[i][3].grid()
  ax[i][4].grid()
  i+=1

In [ ]:
#вероятности для аска + средний экстремум и отклонение этих экстремумов
insts = ['BTCBUSD','BTCUSDT','ETHUSDT_210625','ETHUSDT','BTCUSDT_210625']
Pr = {}
dextr = {}
for p in insts:
  Pr[p] = []
  dextr[p] = {'mean':[],'std':[],'median':[]}
  for k in np.arange(0,3,0.5):
    pn = 0
    p0 = 0
    delta_extremums = []
    for i in range(len(orderbooks[p])-1):
      if orderbooks[p][8].iloc[i] >= ask_meanvols[p]+k*ask_stdvols[p]:
        j = i+1
        #orderbooks[p][5].iloc[i]
        while orderbooks[p][7].iloc[j]==orderbooks[p][7].iloc[i] and j<len(orderbooks[p])-2:
          j+=1
        if orderbooks[p][7].iloc[j]<orderbooks[p][7].iloc[i]:
          pn+=1
          while orderbooks[p][7].iloc[j]<=orderbooks[p][7].iloc[j-1] and j<len(orderbooks[p])-2:
            j+=1
          delta_extremums.append(-orderbooks[p][7].iloc[j-1]/orderbooks[p][7].iloc[i]+1)
        else:
          p0+=1
    dextr[p]['mean'].append(np.mean(delta_extremums))
    dextr[p]['median'].append(np.median(delta_extremums)) 
    dextr[p]['std'].append(np.std(delta_extremums))
    Pr[p].append(pn/(p0+pn))
    print(p,k,":",pn/(p0+pn), dextr[p]['mean'][-1])

# Графики(аск)

In [ ]:
i = 0
fig, ax = plt.subplots(len(insts),5)
fig.set_size_inches(25,4*len(insts))
for p in insts:
  ax[i][0].plot(np.arange(0,3,0.5), Pr[p])
  ax[i][1].plot(np.arange(0,3,0.5), dextr[p]['mean'])
  ax[i][2].plot(np.arange(0,3,0.5), dextr[p]['std'])
  ax[i][3].plot(np.arange(0,3,0.5), dextr[p]['median'])
  ax[i][4].plot(np.arange(0,3,0.5), count_vols[p])
  ax[i][0].set_title(p+' probability')
  ax[i][1].set_title(p+' mean extr')
  ax[i][2].set_title(p+' std extr')
  ax[i][3].set_title(p+' median extr')
  ax[i][4].set_title(p+' vol ratio')
  ax[i][0].grid()
  ax[i][1].grid()
  ax[i][2].grid()
  ax[i][3].grid()
  ax[i][4].grid()
  i+=1